# Import

In [49]:
import os
import sys
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import AllChem

from IPython.core.display import display
from IPython.display import SVG, HTML
from rdkit.Chem.Draw import IPythonConsole
# IPythonConsole.ipython_useSVG = True  # Change output to SVG
# IPythonConsole.drawOptions.addAtomIndices = True

sys.path.append('../src/reactivityQM/')
from molecule_drawer import generate_structure, generate_output_tables, html_output

In [17]:
# config
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({'font.size': 22})

# Change Matplotlib font to Helvetica
import matplotlib as mpl
import matplotlib.font_manager as fm
from matplotlib.legend_handler import HandlerTuple

mpl.rcParams['font.family'] = 'Helvetica'
fm.findfont("Helvetica", fontext="ttx", rebuild_if_missing=False)

'/System/Library/Fonts/Helvetica.ttc'

In [32]:
import py3Dmol
from ipywidgets import interact,fixed,IntSlider
import ipywidgets
def draw3d(
    mols,
    width=400,
    height=400,
    Hs=True,
    confId=-1,
    multipleConfs=False,
    atomlabel=False,
):
    try:
        p = py3Dmol.view(width=width, height=height)
        if type(mols) is not list:
            mols = [mols]
        for mol in mols:
            if multipleConfs:
                for conf in mol.GetConformers():
                    mb = Chem.MolToMolBlock(mol, confId=conf.GetId())
                    p.addModel(mb, "sdf")
            else:
                if type(mol) is str:
                    if os.path.splitext(mol)[-1] == ".xyz":
                        xyz_f = open(mol)
                        line = xyz_f.read()
                        xyz_f.close()
                        p.addModel(line, "xyz")
                else:
                    mb = Chem.MolToMolBlock(mol, confId=confId)
                    p.addModel(mb, "sdf")
        p.setStyle({"sphere": {"radius": 0.4}, "stick": {}})
        if atomlabel:
            p.addPropertyLabels("index")  # ,{'elem':'H'}
        p.zoomTo()
        p.update()
        # p.show()
    except:
        print("py3Dmol, RDKit, and IPython are required for this feature.")

In [33]:
def get_atom_idx(mol, index=1):
  for i,a in enumerate(mol.GetAtoms()):
    if a.GetAtomMapNum() == index:
      return i

# Load data

In [40]:
pkl_files = glob.glob(f'/Users/nicolairee/Desktop/steno/PhD/git/ReactivityQM/submitit_results/testmol/*result.pkl')


calc_failed = []
df = pd.read_pickle(pkl_files[0])[-1]
for f in pkl_files[1:]:
    d = pd.read_pickle(f)
    if d[0] == 'error':
        calc_failed.append(f)
        continue
    df = df.append(d[-1], ignore_index=True, sort=False)

df = df.sort_values(by=['name'], ascending=True)
df.shape[0]

7

# Draw Output

In [71]:
for idx, row in df.iterrows():

    name = row['name']
    rdkit_smiles = row['smiles']
    rdkit_mols = [Chem.MolFromSmiles(smi) for smi in rdkit_smiles.split('.')]

    elec_sites_list = row['elec_sites']
    elec_names_list = row['elec_names']
    MAA_values_list = row['MAA_values']
    MAA_calc_logs = row['MAA_calc_logs']
    nuc_sites_list = row['nuc_sites']
    nuc_names_list = row['nuc_names']
    MCA_values_list = row['MCA_values']
    MCA_calc_logs = row['MCA_calc_logs']

    result_svg = generate_structure(rdkit_mols, elec_sites_list, MAA_values_list, nuc_sites_list, MCA_values_list, molsPerRow=4)
    
    df_elec = generate_output_tables(rdkit_mols, elec_names_list, MAA_values_list, elec_sites_list, MAA_calc_logs, MAA_or_MCA='MAA')
    df_nuc = generate_output_tables(rdkit_mols, nuc_names_list, MCA_values_list, nuc_sites_list, MCA_calc_logs, MAA_or_MCA='MCA')

    result_output = html_output(rdkit_smiles, result_svg, df_elec, df_nuc)
    
    result_output = result_output.replace("""body {\n                font-family: Arial, sans-serif;\n                background-color: #f7f7f7;\n                margin: 0;\n                padding: 20px;\n                }\n\n                """, """body {\n                font-family: Arial, sans-serif;\n                }\n\n                """)
    result_output = result_output.replace('href="', 'href="../calculations/') #Change path of "Show File" to make it work from this .ipynb file. OBS! The .html file must be located in "ReactivityQM/calculations"
    display(HTML(result_output))

    break

Atom ID,MAA Value [kJ/mol],"Error Log (Reactant, Product)",Type
2,241.87,"None, Connectivity",Ester
14,203.06,"None, None",Double bond
6,159.37,"None, None",Double bond
12,151.88,"None, None",Double bond
4,149.31,"None, None",Double bond
13,28.99,"None, None",Double bond
3,12.72,"None, None",Double bond
5,-10.35,"None, None",Double bond
Atom ID,MCA Value [kJ/mol],"Error Log (Reactant, Product)",Type
10,478.44,"None, None",Amine


In [73]:
mols = [next(Chem.SDMolSupplier('../calculations/'+calc_log.split(', ')[1].split('"')[1], sanitize=False)) for calc_log in df_elec['Error Log (Reactant, Product)'].tolist()] # Show MAA products
# mols = [next(Chem.SDMolSupplier('../calculations/'+calc_log.split(', ')[1].split('"')[1], sanitize=False)) for calc_log in df_nuc['Error Log (Reactant, Product)'].tolist()] # Show MCA products

def mol_viewer(idx):
    return draw3d(mols[idx])

interact(mol_viewer, idx=ipywidgets.IntSlider(min=0,max=len(mols)-1, step=1))

interactive(children=(IntSlider(value=0, description='idx', max=7), Output()), _dom_classes=('widget-interact'…

<function __main__.mol_viewer(idx)>

# Show SMILES

In [9]:
def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None

In [31]:
@interact
def smi2viewer(SMILES='COC(=O)c1nc(C(C)(C)N)[nH]c(=O)c1O'):
    try:
        conf = smi2conf(SMILES)
        return draw3d(conf).show()
    except:
        return None

interactive(children=(Text(value='COC(=O)c1nc(C(C)(C)N)[nH]c(=O)c1O', description='SMILES'), Output()), _dom_c…